In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from pathlib import Path
import numpy as np
import PIL
import PIL.Image
import json
from tqdm.auto import tqdm

#change working directory to root
ROOT_DIR = os.getcwd()
while os.path.basename(ROOT_DIR) != 'VisIrNet':
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR,'..'))
sys.path.insert(0,ROOT_DIR)
os.chdir(ROOT_DIR)

ROOT_DIR = Path(ROOT_DIR)

print(tf.__version__)
devices = tf.config.list_physical_devices('GPU')
print("len(devices): ", len(devices))
print(f"available GPUs: {devices}");


2.14.0
len(devices):  1
available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


/home/nalain/miniconda3/envs/VisIrNet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## gpu setup 
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
###

1 Physical GPUs, 1 Logical GPUs


**Configurations**

In [3]:
# config file to load will be passed as an argument
# get run parameters

import argparse 

parser = argparse.ArgumentParser()
parser.add_argument('--config-file', 
                        action = "store", 
                        dest = "config_file",
                        default = "vedai_default_config.json",
                        help = 'specify config file to load')

input_arguments = parser.parse_args([])

from Tools.configurations_parser import ConfigurationParser
# load configurations
configs = ConfigurationParser.getConfigurations(configs_path = 'configs', 
                                                config_file = str(input_arguments.config_file))


# print configurations
ConfigurationParser.printConfigurations(configs)


[INFO] reading configurations from configs/vedai_default_config.json
************************* Configurations *************************
	 dataset: VEDAI
	 TrainFirstStage: True
	 TrainSecondStage: True
	 B_R_uuid: 61a2b4810a9f4e9ba7a084652436f0ea
	 BATCH_SIZE: 2
	 SHUFFLE_BUFFER_SIZE: 1000
	 RGB_INPUTS_SHAPE: [192, 192, 3]
	 IR_INPUTS_SHAPE: [128, 128, 3]
	 B_STACK_COUNT: 1
	 R_STACK_COUNT: 1
	 OUTPUT_CHANNELS_PER_BLOCK: 3
	 REGRESSION_INPUT_SHAPE: [192, 192, 6]
	 REGRESSION_OUTPUT_SHAPE: 8
	 B_initial_learning_rate: 0.01
	 B_decay_steps: 1000
	 B_decay_rate: 0.96
	 B_NUM_EPOCHS: 2
	 B_from_checkpoint: None
	 B_save_path: models/VEDAI
	 B_save_as: featureEmbeddingBackBone
	 B_save_frequency: 1
	 B_save_hard_frequency: 20
	 R_initial_learning_rate: 0.001
	 R_decay_steps: 1000
	 R_decay_rate: 0.96
	 R_NUM_EPOCHS: 2
	 R_featureEmbeddingBackBone: latest
	 R_from_checkpoint: None
	 R_save_path: models/VEDAI
	 R_save_as: regressionHead
	 R_save_frequency: 1
	 R_save_hard_frequency: 10
	 R_pr

**Dataloaders**

In [4]:
import data_setup

train_dataloader,test_dataloader = data_setup.create_dataloaders(dataset=configs.dataset, 
                                                                BATCH_SIZE=configs.BATCH_SIZE,
                                                                SHUFFLE_BUFFER_SIZE=configs.SHUFFLE_BUFFER_SIZE
                                                                )

len(train_dataloader), len(test_dataloader)
#

[INFO] loading train dataset
[INFO] train _dataset:  8722
[INFO] loading val dataset
[INFO] val _dataset:  3738
dataset: VEDAI
BATCH_SIZE: 2
SHUFFLE_BUFFER_SIZE: 1000
train_dataloader: 4361
test_dataloader: 1869


(4361, 1869)

## **Model**

In [5]:
import model_setup
import Utils

featureEmbeddingBackBone = model_setup.getFeatureEmbeddingBackBone(rgb_inputs_shape=configs.RGB_INPUTS_SHAPE,
                                                        ir_inputs_shape=configs.IR_INPUTS_SHAPE,
                                                        output_channels_per_block=configs.OUTPUT_CHANNELS_PER_BLOCK,
                                                        blocks_count=configs.B_STACK_COUNT,
                                                        )

# regressionHead= model_setup.getRegressionHead(input_shape=configs.REGRESSION_INPUT_SHAPE,
#                                                 output_size=configs.REGRESSION_OUTPUT_SHAPE,
#                                                 blocks_count=configs.R_STACK_COUNT,
#                                                 )


**Visualize and save model structures**


## **Training**


**first stage**

In [6]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=configs.B_initial_learning_rate,
                                                                decay_steps=configs.B_decay_steps,
                                                                decay_rate=configs.B_decay_rate,
                                                                staircase=True)


# Setup optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


In [7]:
model = featureEmbeddingBackBone

In [8]:
import Tools.loss_functions as loss_functions
import Tools.datasetTools as DatasetTools


In [10]:
model.compile(optimizer=optimizer)

for epoch in tqdm(range(2)):
    
    for i, batch in enumerate(train_dataloader.take(256)):
        
        input_images, template_images, labels,_instances = batch
        
        tf.config.run_functions_eagerly(True)
        gt_matrix = DatasetTools.get_ground_truth_homographies(labels)
        tf.config.run_functions_eagerly(False)
        
        warped_inputs, _ = DatasetTools._get_warped_sampled(images = input_images,  homography_matrices = gt_matrix)

        ###    warped_inputs

        # warped_inputs.shape, tf.reduce_max(warped_inputs), tf.reduce_min(warped_inputs)

        with tf.GradientTape() as tape:
                        #persistent=True
                        rgb_fmaps , ir_fmaps = model.call((input_images, template_images), training=True)
                        # total_loss = tf.constant(0.0)
                        tape.watch(rgb_fmaps)
                        tape.watch(ir_fmaps)
                        
                        
                        warped_fmaps,_ = DatasetTools._get_warped_sampled( images = rgb_fmaps, 
                                                                            homography_matrices = gt_matrix)
                        
                        tape.watch(warped_fmaps)
                        
                                        
                        total_loss , detailed_batch_losses = loss_functions.get_losses_febackbone( warped_inputs,
                                                                                                    template_images,
                                                                                                    warped_fmaps,
                                                                                                    ir_fmaps)
                        
                        
                    
                    
        # get gradients and backpropagate                
        all_parameters= model.trainable_variables
        grads = tape.gradient(total_loss, all_parameters)#,unconnected_gradients=tf.UnconnectedGradients.ZERO

        grads_are_safe = np.array([ tf.math.is_finite(g).numpy().all() for g in grads ]).all()
        
        assert grads_are_safe, F"Gradients are not safe at iteration: {i}"
        #backpropagate

        grads = [tf.clip_by_value(i,-0.1,0.1) for i in grads] 
        optimizer.apply_gradients(zip(grads, all_parameters))
        
        detailed_batch_losses = {str(i): k.numpy() for i, k in detailed_batch_losses.items()}
        loss_message = " | ".join([str(str(i)+ " : " + str(k)) for i,k in detailed_batch_losses.items()])
                
        # print(f"{i}:: shape: {warped_fmaps.shape} | max: {tf.reduce_max(warped_fmaps)} | min: {tf.reduce_min(warped_fmaps)}| grads_are_safe: {grads_are_safe} --- loss: {total_loss.numpy()}| {loss_message}")
        # print(f"{i}:: shape: {warped_fmaps.shape} | max: {tf.reduce_max(warped_fmaps)} | min: {tf.reduce_min(warped_fmaps)}| grads_are_safe: {grads_are_safe} --- loss: {total_loss.numpy()}")



  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [02:07<00:00, 63.73s/it]


In [12]:
total_loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.07401132>

: 